# Voting Results by Geography Table Exporter

In [6]:
import os
import sqlalchemy
import pandas as pd
import geopandas as gpd

In [9]:
county_fips = 453
election_year = '22'
election_type = 'g'

In [2]:
db_filepath = os.path.join('data', 'databases', 'texas_legislative_council', 'texas_legislative_council_election_dataset_2022.db')
sql_engine = sqlalchemy.create_engine('sqlite:///' + db_filepath)

In [3]:
df = pd.read_sql('election_returns', sql_engine)

In [8]:
df[df['FIPS'] == county_fips]

,index,County,FIPS,VTD,cntyvtd,vtdkeyvalue,Office,Name,Party,Incumbent,Votes
82311,635664,Travis,453,0100,4530100,8706,U.S. Rep 35,Casar,D,N,1415
82312,635665,Travis,453,0101,4530101,8707,U.S. Rep 35,Casar,D,N,1220
82313,635666,Travis,453,0102,4530102,8708,U.S. Rep 35,Casar,D,N,1601
82314,635667,Travis,453,0103,4530103,8709,U.S. Rep 35,Casar,D,N,1091
82315,635668,Travis,453,0104,4530104,8710,U.S. Rep 37,Doggett,D,Y,1976
...,...,...,...,...,...,...,...,...,...,...,...
114398,651705,Travis,453,0392,4530392,8935,Travis JP 2,David,L,N,538
114399,651706,Travis,453,0393,4530393,8936,Travis JP 2,David,L,N,673
114400,651707,Travis,453,0394,4530394,8937,Travis JP 2,David,L,N,727
114401,651708,Travis,453,0395,4530395,8938,Travis JP 2,David,L,N,559
